In [1]:
import os
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import DirectoryLoader



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
pdf_loader = DirectoryLoader(r"C:\Major_Project\NITT_ChatBot\data", glob="*.pdf", loader_cls=PyPDFLoader)
pdf_docs = pdf_loader.load()

In [3]:
web_loader = WebBaseLoader(["https://www.nitt.edu/"])
web_docs = web_loader.load()


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

pdf_chunks = text_splitter.split_documents(pdf_docs)
web_chunks = text_splitter.split_documents(web_docs)

all_chunks = pdf_chunks + web_chunks

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Load free embedding model from Hugging Face
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")

# Create vector DB using Chroma
vector_db = Chroma.from_documents(all_chunks, embeddings, persist_directory="./chroma_db")

vector_db.persist()  # Save embeddings
retriever = vector_db.as_retriever(search_type="similarity",search_kwargs={"k": 5})  # Convert to retriever

C:\Users\Sweety\AppData\Local\Temp\ipykernel_12104\2383269725.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
c:\ProgramData\anaconda3\envs\llmapp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Sweety\AppData\Local\Temp\ipykernel_12104\2383269725.py:10: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.p

In [6]:

from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA, StuffDocumentsChain
from langchain.schema.runnable import RunnablePassthrough

# Define LLM
os.environ["GROQ_API_KEY"] = "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"
llm = ChatGroq(temperature=0.4, max_tokens=500, streaming=True)

prompt_template = PromptTemplate.from_template(
    "Use the following retrieved context to answer the question: {context} \n\nQuestion: {question}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt_template)

document_prompt = PromptTemplate(
    input_variables=["context"],
    template="{context}"  
)


def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])  

document_chain = StuffDocumentsChain(
    llm_chain=llm_chain, 
    document_prompt=document_prompt,  
    document_variable_name="context")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt_template} 
)

C:\Users\Sweety\AppData\Local\Temp\ipykernel_12104\84315260.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_template)
C:\Users\Sweety\AppData\Local\Temp\ipykernel_12104\84315260.py:27: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  document_chain = StuffDocumentsChain(


In [7]:
def chatbot_response(query):
    response = qa_chain.run(query)
    return response


In [9]:
while True:
    user_input = input("\nAsk your college-related query (type 'exit' to quit): ")
    if user_input.lower() in ["exit", "quit"]:
        break
    answer = chatbot_response(user_input)
    print("\nChatbot:", answer)


Chatbot: Based on the provided context, the admission process for the M.Sc. Computer Science program in 2024-25 would be handled through the CCMN 2024 (Centralized Counseling for M.Sc./M.Sc.(Tech.) JAM/GATE). There is no specific information about additional requirements or steps for the M.Sc. Computer Science program, so it is assumed that the general admission process for PG programs applies.

Chatbot: The retrieved context does not provide the fee structure specifically for the M.Sc. program. It includes the fee structure for the MA and MCA programs, as well as general fees such as computer fees, internet fees, library fees, etc. However, there is no mention of the M.Sc. program or its associated fees.

Chatbot: Based on the provided context, the fee structure for the PG program is as follows:

1. Computer Fee: 2650 - 2900
2. Internet Fee: 1100 - 1200
3. Library fee: 4950 - 5500
4. Examination fee: 2650 - 2900
5. Registration-Enrolment fee: 550 - 600
6. Association and Cultural fee